In [1]:
# libraries 

from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
import time
from selenium.webdriver.chrome.service import Service

**Job Board list Scraping**

In [5]:
s = Service(r"C:\Users\Hebert Palmera\Downloads\chromedriver-win64 (1)\chromedriver-win64\chromedriver.exe")

In [3]:

driver = webdriver.Chrome(service=s)

driver.get("https://www.linkedin.com/jobs/search/?currentJobId=3883746336&geoId=103644278&keywords=Arrive%20Logistics&location=United%20States&origin=JOB_SEARCH_PAGE_LOCATION_AUTOCOMPLETE&refresh=true")

scroll_pause_time = 2 
scroll_height = 0

while True:

    #Current scroll height
    prev_scroll_height = scroll_height
    scroll_height = driver.execute_script("return document.body.scrollHeight")

    # Small scroll up
    driver.execute_script("window.scrollBy(0, -100);")

    # wait
    time.sleep(0.5)

    # scroll down the page
    driver.execute_script(f"window.scrollTo(0, {scroll_height});")


    # wait 
    time.sleep(scroll_pause_time)

    try:
        see_more_button = driver.find_element(By.CSS_SELECTOR, f'button[data-tracking-control-name="infinite-scroller_show-more"]')
        if see_more_button:
            see_more_button.click()
    except:
        print('Could find the button')

    # no more new listing are loaded, break the loop
    if scroll_height == prev_scroll_height:
        break



jobs_listing = driver.find_elements(By.XPATH, '//*[@id="main-content"]/section[2]/ul/li')


with open('job_listing.csv','w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Job Title', 'Company Name', 'Address', 'Job Link']) #header row
    for listing in jobs_listing:
        job_title_element = listing.find_element(By.CLASS_NAME, 'base-search-card__title')
        job_title = job_title_element.text

        company_element = listing.find_element(By.CLASS_NAME, 'base-search-card__subtitle')
        company = company_element.text

        address_element = listing.find_element(By.CLASS_NAME, 'job-search-card__location')
        address = address_element.text

            # Check if job link is available
        try:
            link_element = listing.find_element(By.CLASS_NAME, 'base-card__full-link')
            job_link = link_element.get_attribute('href')
        except:
            job_link = "N/A"



        print("Job Title:", job_title)
        print("Company:", company)
        print("Address:", address)
        #print("Salary:", salary)
        print("Link:", job_link)
        print("-" * 50)

        csv_writer.writerow([job_title, company, address,job_link])

#driver.quit()

Could find the button
Could find the button
Job Title: Associate Research Analyst
Company: Arrive Logistics
Address: Austin, Texas Metropolitan Area
Link: https://www.linkedin.com/jobs/view/associate-research-analyst-at-arrive-logistics-3883746336?position=1&pageNum=0&refId=SuUrkxGbQ%2Fm%2F0aq6T3T8xQ%3D%3D&trackingId=8oLlhd0%2FMMJdNxHlWOGZOw%3D%3D&trk=public_jobs_jserp-result_search-card
--------------------------------------------------
Job Title: Business Development Representative
Company: Arrive Logistics
Address: Austin, Texas Metropolitan Area
Link: https://www.linkedin.com/jobs/view/business-development-representative-at-arrive-logistics-3888070353?position=2&pageNum=0&refId=SuUrkxGbQ%2Fm%2F0aq6T3T8xQ%3D%3D&trackingId=IMZ7aQLiwC9FTJU%2BJgHuOQ%3D%3D&trk=public_jobs_jserp-result_search-card
--------------------------------------------------
Job Title: Carrier Sales Representative
Company: Arrive Logistics
Address: Columbus, Ohio Metropolitan Area
Link: https://www.linkedin.com/job

**Scraping more details from the job's description**

In [6]:
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

# Initialize ChromeDriver

driver = webdriver.Chrome(service=s)

# Read CSV file containing job details and URLs
df = pd.read_csv(r"job_listing.csv")

# Function to extract expected salary from URL
def extract_expected_salary(url):
    max_attempts = 3
    for attempt in range(max_attempts):
        time.sleep(3)
        try:
            driver.get(url)
            expected_salary_element = driver.find_element(By.CLASS_NAME, 'salary.compensation__salary')
            expected_salary = expected_salary_element.text
            return expected_salary
        except NoSuchElementException:
            print(f"Attempt {attempt + 1}: Expected salary not found. Retrying...")
            time.sleep(2)  # Pause for a moment before retrying

    return "N/A"

# Extract expected salary for each job listing
df['Expected Salary'] = df['Job Link'].apply(extract_expected_salary)

# Save updated job details to CSV
df.to_csv('job_details_with_salary.csv', index=False)

# Close the ChromeDriver session
driver.quit()

Attempt 1: Expected salary not found. Retrying...
Attempt 2: Expected salary not found. Retrying...
Attempt 3: Expected salary not found. Retrying...
Attempt 1: Expected salary not found. Retrying...
Attempt 2: Expected salary not found. Retrying...
Attempt 3: Expected salary not found. Retrying...
Attempt 1: Expected salary not found. Retrying...
Attempt 2: Expected salary not found. Retrying...
Attempt 3: Expected salary not found. Retrying...
Attempt 1: Expected salary not found. Retrying...
Attempt 2: Expected salary not found. Retrying...
Attempt 3: Expected salary not found. Retrying...
Attempt 1: Expected salary not found. Retrying...
Attempt 2: Expected salary not found. Retrying...
Attempt 3: Expected salary not found. Retrying...
Attempt 1: Expected salary not found. Retrying...
Attempt 2: Expected salary not found. Retrying...
Attempt 3: Expected salary not found. Retrying...
Attempt 1: Expected salary not found. Retrying...
Attempt 2: Expected salary not found. Retrying...
